<a href="https://colab.research.google.com/github/archei2500/overdubbing_courses/blob/main/video_dubbing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> **Добро пожаловать в интерактивную систему для озвучивания видео!**

> Здесь вы можете:
*   выполнить синтез речи с помощью 5 различных инструментов на основе файла со структурой SRT;
*   протестировать некоторые инструменты для синтеза речи и получить рекомендации по их выбору для вашей ситуации;
*   переозвучить видео (заменить аудиодорожку с речью на синтезированную) несколькими способами.

> Пожалуйста, следуйте инструкциям.

> Последовательность работы с блокнотом подразумевается такой:
1.   Для начала перейдите к первому разделу и запустите ячейку. В среду установится всё необходимое для работы. *Это нужно сделать при любом сценарии работы*.
2.   Перейдите ко второму разделу (разверните его, как и следующие): загрузите в систему файл, имеющий структуру SRT (т. е. субтитры, с временными интервалами), для синтеза речи, видео, которое будет переозвучиваться, и образец голоса для клонирования (если оно требуется - РЕКОМЕНДУЕТСЯ ФОРМАТ WAV) любым удобным способом из предложенных. Необязательно загружать всё сразу. *Система уведомит вас, если вы забудете что-то загрузить и попытаетесь выполнить код, где это нужно.*
3.   Перейдите к синтезу речи. Для начала вы можете получить рекомендации по выбору инструмента для синтеза во второй ячейке раздела. Потребуется указать ваши предпочтения (язык синтеза и другие параметры). Затем на их основе вам будет выдан список возможных инструментов, моделей и голосов. Некоторые из инструментов доступны для тестирования в третьей ячейке раздела. После того, как вы определитесь с выбором, заполните поля первой ячейки и запустите её. После этого будут синтезированы фразы по отдельности. Вы можете скачать zip-архив с ними.
4. Перейдите к клонированию голоса и выберите инструмент. Вы также получите wav-файлы с фразами, но с изменённым голосом, которые вы также сможете скачать как zip-архив.
5. Теперь синтезированная речь может быть наложена на видео вместо оригинальной. Выберите один из двух вариантов с их опциями в зависимости от того, что вам нужно: достаточно простого соответствия фраз времени их начала, нужно подгонять фразы под временные отрезки, или видео должно быть чётко синхронизировано с новыми фразами при их исходной равной скорости. *Для последнего варианта необходимо запустить ячейку с нарезкой видео на фрагменты.* В результате вы получите переозвученное видео.

> *Примечание 1: Вы можете загрузить синтезированные фразы и фрагменты видео в систему в разделе загрузок в виде zip-архивов. Это позволит вам возвращаться к процессу в удобное для вас время без необходимости проделывать всё с нуля.*

> ***Желательно подключаться к среде с GPU, если планируете выполнять этап синтеза речи - особенно с Coqui.***

# Запустить перед началом работы

In [ ]:
#!pip install moviepy
!pip install pydub
!pip install ffmpeg-python
!pip install iso639
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips
from moviepy.video.fx.accel_decel import accel_decel
import re
from IPython.display import clear_output, Audio, display
from pydub import AudioSegment
import os
from google.colab import files
import subprocess
import ffmpeg
import torch
import locale # для восстановления кодировки
import iso639 # для кодов языков
import cv2

times = [] # для корректировки субтитров в конце


def str_to_time(s):
  return int(s[6:8]) + 60 * int(s[3:5]) + 3600 * int(s[:2]) + float('0.' + s[9:12])


def time_to_str(time):
  hours = int(time // 3600)
  minutes = int((time - 3600 * hours) // 60)
  seconds = time - 60 * minutes
  str_sec = str(round(seconds, 3)).replace('.', ',')
  return str(hours).zfill(2) + ':' + str(minutes).zfill(2) + ':' + str_sec[:str_sec.find(',')].zfill(2) + ',' + str_sec[str_sec.find(',') + 1:].ljust(3, '0')


def extract_number(s):
  match = re.search(r'\d+', s) # регулярное выражение \d+ означает "одна или более цифр подряд"
  if match:
    return int(match.group())
  else:
    return 0


def add_pause(old_path, aud_path, dur):
  # генерация тишины (в миллисекундах)
  silence = AudioSegment.silent(duration = dur * 1000)
  audio = AudioSegment.from_file(old_path)
  audio += silence
  audio.export(aud_path, format = "wav")


def speedup(path, speed):
  filter = "atempo=" + str(speed)
  command = "ffmpeg -i " + path + " -filter:a " + filter + " /content/changed.wav"
  process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  output, error = process.communicate()
  os.remove(path)
  os.rename("/content/changed.wav", path)


def check_duration(old_path, aud_path, start, end, slow_aud, limit):
  duration = (end - start) * 1000 # в миллисекундах
  # измеряем продолжительность синтезированного фрагмента
  audio = AudioSegment.from_file(path)
  if len(audio) != duration: # если аудио получилось больше или меньше требуемой продолжительности
    # нужно его ускорить и замедлить или добавить паузу (в зависимости от режима)
    coef = len(audio) / duration # коэффициент растяжения или сжатия
    if coef < 1:
      if not slow_aud: # не замедляем аудио
        add_pause(old_path, aud_path, (duration - len(audio)) / 1000)
      else:
        if limit:
          if coef >= 0.9: # замедление будет выполнено, только если аудио не станет медленнее этого значения
            !cp $old_path $aud_path
            speedup(aud_path, coef)
        else:
          !cp $old_path $aud_path
          speedup(aud_path, coef)
    else:
      # ограничение на ускорение в случае с аудио не накладываем, т. к. в видеофрагмент важно уложиться
      !cp $old_path $aud_path
      speedup(aud_path, coef)
  else: # Всё равно копируем в новую папку
    !cp $old_path $aud_path


def getpreferredencoding(do_setlocale = True):
    return "UTF-8"


def video_duration(filename):
  video = cv2.VideoCapture(filename)

  fps = video.get(cv2.CAP_PROP_FPS)
  frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
  duration = frame_count / fps

  video.release()

  return duration


def check_vid_duration(aud_path, vid_path, limit):
  # получение продолжительности видео
  audio = AudioSegment.from_file(aud_path)
  video = VideoFileClip(vid_path).without_audio()
  times.append(len(audio) / 1000)
  if abs(len(audio) / 1000 - video.duration) > 0.05: # аудио короче или длиннее видео на более чем 50 мс
    coef = round(video.duration / (len(audio) / 1000), 3)
    if limit:
      # проверка коэффициента, замедление или усорение будет выполнено, только если коэффициент в данных пределах
      if coef <= 1.1 and coef >= 0.9:
        video = accel_decel(video, video.duration / coef, abruptness = 0)
        os.remove(vid_path)
        video.write_videofile(vid_path, threads = 4)
      else: # иначе работа с самим аудио
        if coef > 1: # если видео собирались ускорить, к аудио добавляем паузу
          pause_len = video.duration - (len(audio) / 1000) # в с
          add_pause(aud_path, aud_path, pause_len)
        else: # если видео собирались замедлить, то ускоряем аудио
          speedup(aud_path, 1 / coef)
          audio = AudioSegment.from_file(aud_path)
          times.pop()
          times.append(len(audio) / 1000)
    else:
      video = accel_decel(video, video.duration / coef, abruptness = 0)
      os.remove(vid_path)
      video.write_videofile(vid_path, threads = 4)


def form_boundary(time, clone_path):
  ok = True
  pos = time.find(':')
  if pos != -1:
    buf = time[: pos]
    if not (len(buf) > 0 and len(buf) < 3 and buf.isdigit() and int(buf) > 0):
      ok = False
    buf = time[pos + 1 :]
    if not (len(buf) > 0 and len(buf) < 3 and buf.isdigit() and int(buf) > 0):
      ok = False
    if ok:
      buf = int(time[: pos]) * 60 + int(time[pos + 1 :])
      dur = len(AudioSegment.from_file(clone_path))
      if buf > dur: # если больше продолжительности видео
        ok = False
        print("Введённое вами время превышает продолжительность аудиофайла!")
  else:
    ok = False

  return ok, buf


def choice_silero_model(lang, gender='female'):
  ver = '3-4'
  voice = None # для тех языков, где нет выбора голосов
  indic_languages = ['bengali', 'gujarati', 'hindi', 'kannada', 'malayalam',
                     'manipuri', 'rajasthani', 'tamil', 'telugu']

  if lang == 'russian':
    model_id = 'v4_ru'
  elif lang == 'english':
    model_id = 'v3_en'
  elif lang == 'german':
    model_id = 'v3_de'
  elif lang == 'spanish':
    model_id = 'v3_es'
  elif lang == 'french':
    model_id = 'v3_fr'
  elif lang == 'bashkir':
    model_id = 'aigul_v2'
    ver = '2'
  elif lang == 'kalmyk':
    model_id = 'v3_xal'
  elif lang == 'tatar':
    model_id = 'v3_tt'
    voice = 'dilyara'
  elif lang == 'uzbek':
    model_id = 'v4_uz'
    voice = 'dilnavoz'
  elif lang == 'ukrainian':
    model_id = 'v4_ua'
  # индийские языки
  elif lang in indic_languages:
    model_id = 'v4_indic'
    if lang != 'manipuri':
      voice = language + '_' + gender
    else:
      voice = 'manipuri_female'
  # кириллические языки
  else:
    model_id = 'v4_cyrillic'

  return model_id, voice, ver


# Функция синтеза для Yandex-Speechkit
def synthesize(text, export_path, voice, role):
  model = model_repository.synthesis_model()

  # настройки синтеза
  model.voice = voice
  model.role = role

  # синтез речи и создание аудио с результатом
  result = model.synthesize(text, raw_format = False)
  result.export(export_path, 'wav')


# Проверка на то, что файл имеет необходимую структуру
def check_txtfile(filename):
  txtfile = open(filename, "r", encoding = "utf-8")
  lines = [''] + txtfile.read().split('\n')
  txtfile.close()
  ok = True
  if (len(lines) % 4 == 0):
    for i in range(0, len(lines), 4):
      if lines[i]:
        ok = False
      if not re.match(r'^\d+$', lines[i + 1]):
        ok = False
      if not re.match(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}', lines[i + 2]):
        ok = False
      if not lines[i + 3]:
        ok = False
  else:
    ok = False
  return ok


def silero_save(audio, path, sample_rate):
  # Преобразуем тензор в массив NumPy
  audio_np = audio.numpy()
  # Нормализуем значения массива NumPy к диапазону [-1, 1]
  audio_np = np.clip(audio_np, -1, 1)
  # Масштабируем значения к диапазону int16 для сохранения в формате WAV
  audio_np_int16 = np.int16(audio_np * 32767)
  # Сохраняем массив NumPy как файл WAV
  wavfile.write(path, sample_rate, audio_np_int16)


# переменные для регулирования процесса загрузки видео и/или аудио
yt_downloaded = False
gd_mounted = False
gtts_installed = False
edge_installed = False
silero_installed = False
yandex_installed = False
coqui_installed = False
vm_crtd = False

path_to_text = '/content/subtitles.srt'
path_to_video = '/content/vid.mp4'
clone_sample = '/content/clone_voice.wav'
path_to_init = '/content/synthesized1'
path_to_synth = '/content/synthesized'

yandex_languages = {'german' : [{'name': 'lea', 'gender' : 'female', 'roles' : []}],
                    'english' : [{'name': 'john', 'gender' : 'male', 'roles' : []}],
                    'hebrew' : [{'name': 'naomi', 'gender' : 'female', 'roles' : ['modern', 'classic']}],
                    'kazakh' : [{'name': 'amira', 'gender' : 'female', 'roles' : []},
                                  {'name': 'madi', 'gender' : 'male', 'roles' : []}],
                    'russian' : [{'name': 'alena', 'gender' : 'female', 'roles' : ['neutral', 'good']},
                                  {'name': 'filipp', 'gender' : 'male', 'roles' : []},
                                  {'name': 'ermil', 'gender' : 'male', 'roles' : ['neutral', 'good']},
                                  {'name': 'jane', 'gender' : 'female', 'roles' : ['neutral', 'good', 'evil']},
                                  {'name': 'madirus', 'gender' : 'male', 'roles' : []},
                                  {'name': 'omazh', 'gender' : 'female', 'roles' : ['neutral', 'evil']},
                                  {'name': 'zahar', 'gender' : 'male', 'roles' : ['neutral', 'good']},
                                  {'name': 'dasha', 'gender' : 'female', 'roles' : ['neutral', 'good', 'friendly']},
                                  {'name': 'julia', 'gender' : 'female', 'roles' : ['neutral', 'strict']},
                                  {'name': 'lera', 'gender' : 'female', 'roles' : ['neutral', 'friendly']},
                                  {'name': 'marina', 'gender' : 'female', 'roles' : ['neutral', 'whisper', 'strict']},
                                  {'name': 'alexander', 'gender' : 'male', 'roles' : ['neutral', 'good']},
                                  {'name': 'kirill', 'gender' : 'male', 'roles' : ['neutral', 'strict', 'good']},
                                  {'name': 'anton', 'gender' : 'male', 'roles' : ['neutral', 'good']}],
                    'uzbek' : [{'name': 'nigora', 'gender' : 'female', 'roles' : []}]
                    }
xtts_languages = ['english', 'spanish', 'french', 'german', 'italian', 'portuguese', 'polish', 'turkish', 'russian', 'dutch', 'czech', 'arabic', 'chinese', 'japanese', 'hungarian', 'korean']
silero_languages = ['Russian', 'Ukrainian', 'Uzbek', 'Avar', 'Bashkir', 'Bulgarian', 'Chechen',
                    'Chuvash', 'Erzya', 'Kalmyk', 'Karachay-Balkar', 'Kazakh', 'Khakas',
                    'Komi-Ziryan', 'Lezghian', 'Mari', 'Mari High', 'Nogai', 'Ossetic', 'Tatar',
                    'Tuvinian', 'Udmurt', 'Yakut', 'Hindi', 'Malayalam', 'Manipuri', 'Bengali',
                    'Rajasthani', 'Tamil', 'Telugu', 'Gujarati', 'Kannada', 'English', 'German',
                    'Spanish', 'French']

clear_output()

# Загрузка текста, видео и/или образца голоса

In [ ]:
# @title Загрузка текста (файла со структурой SRT)

# @markdown Выберите метод загрузки и укажите путь к файлу, если привязываете Google Диск:
# @markdown ### Способ загрузки
upload_method = "С устройства" #@param ["С устройства", "Путь к файлу Google Drive"]
#@markdown <font color="orange"> Примечания и советы:

#@markdown <font color="orange"> ``1. При выборе загрузки с устройства нужно будет нажать на кнопку в выходных данных и выбрать файл с устройства.``

#@markdown <font color="orange"> ``2. Обратите внимание: при выборе способа, связанного с Google Диском, система попросит вас дать разрешение на доступ к вашему Google Drive. Затем он будет смонтирован в файловую систему. После этого укажите путь к файлу в соответствующем поле.``

#@markdown ``Введите полный путь к видео на вашем Google Диске (для варианта с загрузкой с Google Drive) `` 👇
path_google_drive = '/content/drive/MyDrive/path_to_subs' #@param {type:"string"}

# Удаление файла, если такой уже был в ФС прежде
if os.path.isfile(path_to_text):
    os.remove(path_to_text)

path_to_text = '/content/subtitles.srt' # путь к файлу по умолчанию

# Видеофайл загружается разными способами
if upload_method == "С устройства":
  uploaded = files.upload()
  for filename in uploaded.keys():
    if not (filename.endswith('.srt') or filename.endswith('.txt')):
      print("Файл должен иметь расширение SRT или TXT!")
      raise SystemExit(0)
    if '/content/' + filename != path_to_text:
      os.rename(filename, path_to_text) # переименование загруженного файла
else:
  if not gd_mounted:
    from google.colab import drive
    drive.mount("/content/drive") # монтирование Google Диска при первом запуске
    gd_mounted = True
  if not os.path.isfile(path_google_drive): # если файл не найден
    print("ERROR: File not found!")
    raise SystemExit(0)
  if not (path_google_drive.endswith('.srt') or path_google_drive.endswith('.srt')):
    print("Файл должен иметь расширение SRT или TXT!")
    raise SystemExit(0)
  !cp $path_google_drive $path_to_text # копирование файла с Google Диска

# Проверка на то, что файл имеет необходимую структуру
if check_txtfile(path_to_text):
  clear_output()
else:
  print('Неверная структура выбранного файла!')

In [ ]:
# @title Загрузка видео
# @markdown Выберите метод загрузки и укажите путь к файлу, если привязываете Google Диск:
# @markdown ### Способ загрузки
upload_method = "С устройства" #@param ["С устройства", "По ссылке с YouTube", "Путь к файлу Google Drive"]
#@markdown <font color="orange"> Примечания и советы:

#@markdown <font color="orange"> ``1. При выборе загрузки с устройства нужно будет нажать на кнопку в выходных данных и выбрать файл с устройства.``

#@markdown <font color="orange"> ``2. Для загрузки с YouTube укажите ссылку в соответствующее поле.``

#@markdown <font color="orange"> ``3. Обратите внимание: при выборе способа, связанного с Google Диском, система попросит вас дать разрешение на доступ к вашему Google Drive. Затем он будет смонтирован в файловую систему. После этого укажите путь к файлу в соответствующем поле.``

#@markdown ``Вставьте ссылку на видеоролик с платформы YouTube в требуемом формате `` 👇
youtube_url = 'https://www.youtube.com/watch?v=YOUTUBE_ID' #@param {type:"string"}

#@markdown ``Введите полный путь к видео на вашем Google Диске (для варианта с загрузкой с Google Drive) `` 👇
path_google_drive = '/content/drive/MyDrive/path_to_vid' #@param {type:"string"}

from urllib import parse as urlparse

# Удаление файла, если такой уже был в ФС прежде
if os.path.isfile(path_to_video):
    os.remove(path_to_video)

path_to_video = '/content/vid.mp4' # путь к видео по умолчанию

# Видеофайл загружается разными способами
if upload_method == "С устройства":
  uploaded = files.upload()
  for filename in uploaded.keys():
    if '/content/' + filename != path_to_video:
      os.rename(filename, path_to_video) # переименование загруженного файла
elif upload_method == "По ссылке с YouTube":
  if not yt_downloaded:
    !pip install yt-dlp # установка при первом запуске
    yt_downloaded = True
  url_data = urlparse.urlparse(youtube_url)
  query = urlparse.parse_qs(url_data.query)
  YOUTUBE_ID = query["v"][0]
  # Загрузка видео с YouTube
  !yt-dlp -f "bestvideo[ext=mp4]+bestaudio[ext=wav]/best[ext=mp4]/best" --output "/content/vid.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
  # Запоминаем путь к файлу
  for file in os.listdir('/content'):
    if file.startswith('vid'):
        path_to_video = os.path.join(directory, file)
else:
  if not gd_mounted:
    from google.colab import drive
    drive.mount("/content/drive") # монтирование Google Диска при первом запуске
    gd_mounted = True
  if not os.path.isfile(path_google_drive): # если файл не найден
      print("ERROR: File not found!")
      raise SystemExit(0)
  !cp $path_google_drive $path_to_video # копирование файла с Google Диска

clear_output() # очистка выходных данных

In [ ]:
# @title Образец голоса для клонирования
# @markdown Клонирование голоса может понадобиться, если для вас важно максимально приближенное звучание голоса к оригиналу.

# @markdown Также это необходимо для XTTSv2 из Coqui TTS

# @markdown ### Выберите метод получения образца голоса для клонирования
clone_method = "С устройства" #@param ["С устройства", "Путь к файлу Google Drive", "Фрагмент из видео"]

#@markdown <font color="orange"> Примечания и советы:

#@markdown <font color="orange"> ``1. При выборе загрузки с устройства нужно будет нажать на кнопку в выходных данных и выбрать файл с устройства.``

#@markdown <font color="orange"> ``2. Обратите внимание: при выборе способа, связанного с Google Диском, система попросит вас дать разрешение на доступ к вашему Google Drive. Затем он будет смонтирован в файловую систему. После этого укажите путь к файлу в соответствующем поле.``

#@markdown <font color="orange"> ``3. Для извлечения образца из видео укажите в соответствующее поле временной интервал.``

#@markdown <font color="red"> ``Если необходимо обрезать загруженный аудиофайл, также укажите интервал в тех же полях.``

#@markdown ``Введите полный путь к видео на вашем Google Диске (для варианта с загрузкой с Google Drive) `` 👇
path_google_drive = '/content/drive/MyDrive/path_to_aud' #@param {type:"string"}

#@markdown ``Введите начало и конец промежутка в формате <минуты>:<секунды> или 0, если не задаёте какую-то или обе границы`` 👇
start_str = "0" #@param {type:"string"}
end_str = "0" #@param {type:"string"}

# Удаление файла, если такой уже был в ФС прежде
if os.path.isfile(clone_sample):
    os.remove(clone_sample)

clone_sample = '/content/clone_voice.wav' # путь к образцу по умолчанию
ok_s = False
ok_e = False

# Разные способы получения образца голоса
if clone_method == "С устройства":
  uploaded = files.upload()
  for filename in uploaded.keys():
    # переименование загруженного файла
    if filename.endswith('.wav'):
      if '/content/' + filename != clone_sample:
        os.rename(filename, clone_sample)
    elif filename.endswith('.mp3'):
      if '/content/' + filename != '/content/clone_voice.mp3':
        os.rename(filename, '/content/clone_voice.mp3')
      clone_sample = '/content/clone_voice.mp3'
  clear_output() # очистка выходных данных
elif clone_method == "Путь к файлу Google Drive":
  if not gd_mounted:
    from google.colab import drive
    drive.mount("/content/drive") # монтирование Google Диска при первом запуске
    gd_mounted = True
  if not os.path.isfile(path_google_drive): # если файл не найден
      print("ERROR: File not found!")
      raise SystemExit(0)
  if path_google_drive.endswith('wav'):
    !cp $path_google_drive $clone_sample # копирование файла с Google Диска
  elif path_google_drive.endswith('mp3'):
    !cp $path_google_drive /content/clone_voice.mp3
    clone_sample = '/content/clone_voice.mp3'
  clear_output()
else:
  if os.path.isfile(path_to_video):
    # извлечение аудиодорожки из видео
    video = VideoFileClip(path_to_video)
    video.audio.write_audiofile('/content/clone_voice.wav')
    clear_output()
  else:
    print('Вы не загрузили видео!')
    raise SystemExit(0)

# обрезание любого аудиофайла
if start_str != '0':
  ok_s, start = form_boundary(start_str) # формирование времени начала в секундах
else:
  start = 0
  ok_s = True
if end_str != '0':
  ok_e, end = form_boundary(end_str) # формирование времени окончания в секундах
else:
  end = 0
  ok_e = True

if ok_s and ok_e:
  # обрезаем аудио (AudioSegment работает с миллисекундами)
  old_audio = AudioSegment.from_file(clone_sample)
  if start != 0 and end != 0:
    extract = old_audio[start * 1000:end * 1000]
  elif start == 0:
    extract = old_audio[:end * 1000]
  elif end == 0:
    extract = old_audio[start * 1000:]
  else:
    extract = old_audio
  # сохраняем
  !rm $clone_sample
  extract.export(clone_sample, format = clone_sample[-3:])

  clear_output()
else:
  print("Вы неправильно ввели значения! Попробуйте ещё раз. Пример в примечании.")

In [ ]:
# @title Дополнительно: загрузка zip-архива с синтезированными фразами

# @markdown Сформируйте zip-архив, содержащий пронумерованные по порядку wav-файлы, и нажмите на кнопку загрузки, которая появится здесь.

!rm -rf /content/synthesized1

# Загрузка zip-архива с wav-файлами
uploaded = files.upload()

# Распаковка архива
for fn in uploaded.keys():
  !unzip {fn} -d /content/speech_downloaded

# Копирование всех файлов wav в буферную папку
buf_fold = '/content/buffer_folder'
os.makedirs(buf_fold, exist_ok=True)

for root, dirs, filess in os.walk('/content/speech_downloaded'):
  for file in filess:
    if file.endswith('.wav'):
     os.rename(os.path.join(root, file), os.path.join(buf_fold, file))

# Удаление распакованной папки
!rm -rf /content/speech_downloaded

# Переименование буферной папки
os.rename(buf_fold, '/content/synthesized1')

clear_output()

In [ ]:
# @title Дополнительно: загрузка zip-архивов с нарезанными фрагментами видео

# @markdown Сформируйте 2 zip-архива, содержащие пронумерованные по порядку mp4-файлы, и нажмите на кнопку загрузки, которая появится здесь.

# @markdown Заметьте, что первый файл должен называться fragments.zip, а второй - int_fragms.zip.

# @markdown Первый содержит фрагменты, в которых присутствует речь, а второй - промежуточные.

!rm -rf /content/fragments
!rm -rf /content/int_fragms

# Загрузка zip-архивов с mp4-файлами
uploaded = files.upload()

# Распаковка архива
for fn in uploaded.keys():
  if fn == 'fragments.zip':
    !unzip {fn} -d /content/fr
  elif fn == 'int_fragms.zip':
    !unzip {fn} -d /content/ifr
  else:
    print('К сожалению, вы назвали один из файлов неправильно. Перепроверьте.')
    !rm -rf /content/fr
    !rm -rf /content/ifr
    raise SystemExit(0)

# Копирование всех файлов wav в буферную папку
buf_fold1 = '/content/buffer_folder1'
buf_fold2 = '/content/buffer_folder2'
os.makedirs(buf_fold1, exist_ok=True)
os.makedirs(buf_fold2, exist_ok=True)

for root, dirs, filenames in os.walk('/content/fr'):
  for file in filenames:
    if file.endswith('.mp4'):
     os.rename(os.path.join(root, file), os.path.join(buf_fold1, file))
for root, dirs, filenames in os.walk('/content/ifr'):
  for file in filenames:
    if file.endswith('.mp4'):
     os.rename(os.path.join(root, file), os.path.join(buf_fold2, file))

# Удаление распакованных папок
!rm -rf /content/fr
!rm -rf /content/ifr

# Переименование буферных папок
os.rename(buf_fold1, '/content/fragments')
os.rename(buf_fold2, '/content/int_fragms')

clear_output()

# Синтез речи

In [ ]:
# @title Синтез речи
# @markdown <font color="green"> *Чтобы получить помощь в выборе инструмента и голосов, а также протестировать голоса, воспользуйтесь ячейками ниже.*

# @markdown Выберите инструмент для синтеза:
tool = 'Coqui TTS' # @param ["Yandex SpeechKit", "Coqui TTS", "gTTS", "Microsoft Edge TTS", "Silero Models"]
# @markdown Введите язык (на английском), на котором будет производиться синтез:
language = 'english' # @param {type: "string"}
if language and language.isalpha():
  language = language.lower()
  lang_capital = language[0].upper() + language[1 :]
else:
  print('Пожалуйста, введите язык синтеза корректно.')
  raise SystemExit(0)
# @markdown Собираетесь ли вы клонировать голос? (для повышения сходства с оригиналом)
cloning = False  # @param {type: "boolean"}
# @markdown Какой голос предпочтительнее?
gender = "мужской"  # @param ["мужской", "женский"]
# @markdown Нужно ли сразу замедлять или ускорять синтезированные фразы? Введите 1, если не надо, и скорость, если надо.
speed_str = "1" #@param {type: "string"}
try:
  speed = float(speed_str)
  if speed <= 0:
    print('Скорость не может быть отрицательным значением!')
    raise SystemExit(0)
except ValueError:
  print('Вы ввели некорректное значение скорости!')
  SystemExit(0)

# @markdown <font color="red"> <font size=4> <center> Дополнительные настройки

# @markdown <font color="orange"> Выберите голос (для Yandex, Edge и Silero):
voice = "john" # @param {type: "string"}
#@markdown <font color="lightblue"> Выберите амплуа голоса (или "нет", если его у модели нет) - для Yandex:
role = "нет" # @param ["нет", "neutral", "good", "evil", "friendly", "strict", "whisper", "modern", "classic"]
# @markdown <font color="purple"> Выберите модель (для Coqui TTS):
model_name = "xtts_v2" #@param ["xtts_v2", "tacotron2-DDC_ph (only english)"]
# @markdown <font color="pink"> Введите ваш ключ API для Yandex SpeechKit:
API_key = "" #@param {type: "string"}
# @markdown <font color="green"> Скачать zip-архив с синтезированной речью?
download_file = False #@param {type: "boolean"}


xtts_inst = False
taco_inst = False
path_to_init = '/content/synthesized1' # синтезированные фразы до добавления к ним пауз
!mkdir /content/synthesized1

# проверка на то, что файл был загружен
if os.path.isfile(path_to_text):
  # Получить устройство
  device = "cuda" if torch.cuda.is_available() else "cpu"

  # ЕСЛИ ВЫБРАЛИ YANDEX
  if tool == "Yandex-Speechkit":
    # Проверки входных данных
    if language not in yandex_languages:
      print('Данный язык не поддерживается Yandex SpeechKit!')
      raise SystemExit(0)
    if voice not in [item['name'] for item in yandex_languages[language]]:
      print('В Yandex SpeechKit нет такого голоса! Посмотрите доступные по ссылке в ячейке ниже.')
      raise SystemExit(0)
    gender = 'male' if gender == 'мужской' else 'female'
    if not cloning and gender != [item['gender'] for item in yandex_languages[language] if item['name'] == voice][0]:
      print('Этот голос имеет другой пол. Попробуйте воспользоваться ячейкой ниже, чтобы выбрать подходящий голос.')
      raise SystemExit(0)
    if not API_key:
      print('Для синтеза речи с помощью Yandex SpeechKit необходим ключ API!')
      raise SystemExit(0)

    if not yandex_installed:
      # установка
      !pip install yandex-speechkit
      from speechkit import model_repository, configure_credentials, creds

      # Аутентификация через API-ключ.
      configure_credentials(
        yandex_credentials=creds.YandexCredentials(
            api_key = API_key
        )
      )
      yandex_installed = True

    # синтез речи с помощью Yandex SpeechKit
    for i in range(0, len(lines), 4):
      export_path = path_to_init + '/' + str(int(i / 4)) + '.wav'
      synthesize(lines[i+3], export_path) # синтез фразы
      if speed != 1:
        speedup(export_path, speed)

  # ЕСЛИ ВЫБРАЛИ COQUI
  elif tool == "Coqui TTS":
    if model_name == "xtts_v2" and not os.path.isfile(clone_sample):
      print('XTTSv2 обязательно нужен файл для клонирования голоса, но вы не загрузили образец! Вернитесь к предыдущему разделу.')
      raise SystemExit(0)
    if model_name == "xtts_v2" and language not in xtts_languages:
      print('Данный язык не поддерживается XTTSv2!')
      raise SystemExit(0)
    if model_name == "tacotron2-DDC_ph (only english)" and language != "english":
      print('tacotron2-DDC_ph поддерживает только английский язык!')
      raise SystemExit(0)

    if not coqui_installed:
      # установка TTS
      !pip install TTS
      from TTS.api import TTS
      coqui_installed = True
    if model_name == "xtts_v2":
      if not xtts_inst:
        # загрузка многоязычной модели
        model_str = "tts_models/multilingual/multi-dataset/" + model_name
        tts = TTS(model_str).to(device)
        xtts_inst = True
      # синтез речи с помощью XTTSv2
      lang_code = iso639.to_iso639_1(lang_capital)
      for i in range(0, len(lines), 4):
        export_path = path_to_init + '/' + str(int(i / 4)) + '.wav'
        # синтез фразы
        tts.tts_to_file(text = lines[i+3], file_path = export_path,
                        speaker_wav = [clone_sample],
                        language = lang_code)
        if speed != 1:
          speedup(export_path, speed)
    else: # tacotron2-DDC_ph
      if not taco_inst:
        model_str = "tts_models/en/ljspeech/" + model_name
        tts = TTS(model_str).to(device)
        taco_inst = True
      # синтез речи с помощью tacotron2-DDC_ph
      for i in range(0, len(lines), 4):
        export_path = path_to_init + '/' + str(int(i / 4)) + '.wav'
        # синтез фразы
        tts.tts_to_file(text = lines[i+3], file_path = export_path)
        if speed != 1:
          speedup(export_path, speed)

    # восстановление кодировки в среде (UTF-8)
    locale.getpreferredencoding = getpreferredencoding

  # ЕСЛИ ВЫБРАЛИ GTTS
  elif tool == "gTTS":
    if not gtts_installed:
      # установка, если не было установлено
      !pip install gTTS
      from gtts import gTTS, lang
      gtts_installed = True
    # проверка наличия языков
    lang_av_ct = 0
    for val in lang.tts_langs().values():
      if lang_capital in val:
        lang_av_ct += 1
    if lang_av_ct > 1:
      print('Найдено несколько подходящих языков:')
      lang_arr = [val for val in lang.tts_langs().values() if lang_capital in val]
      print(', '.join(lang_arr))
      while True:
        try:
          lang_num = int(input('Выберите один из них (введите номер от 1 до ', lang_av_ct, '):'))
          if 1 <= lang_num <= lang_av_ct:  # проверка на то, что число находится в нужном диапазоне
              break
          else:
              print("Вы ввели номер неправильно. Попробуйте снова.")
        except ValueError:
          print("Вы неправильно ввели номер, попробуйте снова.")
      lang_code = {v: k for k, v in lang.tts_langs().items()}.get(lang_arr[lang_num-1]) # инвертирование словаря
    elif not lang_av_ct:
      print('gTTS не поддерживает данный язык!')
      raise SystemExit(0)
    else:
      # получение кода языка из инвертированного словаря, возвращаемого gTTS
      lang_code = {v: k for k, v in lang.tts_langs().items()}.get(lang_capital)
    # синтез речи с помощью gTTS
    for i in range(0, len(lines), 4):
      export_path = path_to_init + '/' + str(int(i / 4)) + '.wav'
      # синтез фразы
      aud_gtts = gTTS(lines[i+3], lang = lang_code)
      aud_gtts.save(export_path)
      if speed != 1:
        speedup(export_path, speed)

  # ЕСЛИ ВЫБРАЛИ EDGE TTS
  elif tool == "Microsoft Edge TTS":
    if not edge_installed:
      # установка
      !pip install edge-tts
      import asyncio
      import edge_tts
      from edge_tts import VoicesManager
      edge_installed = True
    # проверка наличия языка и модели
    lang_code = iso639.to_iso639_1(language) # получение кода языка ISO639-1
    gender = 'Male' if gender == 'мужской' else 'Female'
    if not vm_crtd:
      vm = await VoicesManager.create() # создание объекта голосового менеджера
      vm_crtd = True
    voices = vm.find(Language = lang_code)
    if not voices:
      print('Данный язык не поддерживается Microsoft Edge TTS!')
      raise SystemExit(0)
    v_found = False
    for v in voices: # проверка на то, что выбранный голос есть для такого языка
      if v['ShortName'] == voice:
        if not cloning:
          if v['Gender'] != gender:
            print('Этот голос имеет другой пол. Попробуйте воспользоваться ячейкой ниже, чтобы выбрать подходящий голос.')
            raise SystemExit(0)
        v_found = True
    if not v_found:
      print('Голоса с таким названием нет для выбранного язка!')
      raise SystemExit(0)

    # синтез речи с использованием Edge TTS
    for i in range(0, len(lines), 4):
      export_path = path_to_init + '/' + str(int(i / 4)) + '.wav'
      # синтез фразы
      aud_edge = edge_tts.Communicate(line[i+3], voice)
      await aud_edge.save(export_path)
      if speed != 1:
        speedup(export_path, speed)

  # ЕСЛИ ВЫБРАЛИ SILERO MODELS
  else:
    # проверка на наличие языка
    if lang_capital not in silero_languages:
      print('Данный язык не поддерживается Silero Models!')
      raise SystemExit(0)

    if not silero_installed:
      # установка
      !pip install -q torchaudio omegaconf
      from pprint import pprint
      from omegaconf import OmegaConf
      from scipy.io import wavfile
      import numpy as np
      silero_installed = True
    lang_code = iso639.to_iso639_1(language) # получение кода языка ISO639-1
    gender = 'male' if gender == 'мужской' else 'female'
    # выбор модели
    model_id, speaker, ver = choice_silero_model(language, gender=gender)
    if not voice and ver == '3-4':
      speaker = voice
    model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                          model='silero_tts',
                                          language=lang_code,
                                          speaker=model_id)
    model.to(device)
    # синтез речи с помощью Silero Models
    if ver == '3-4':
      sample_rate = 48000
      for i in range(0, len(lines), 4):
        export_path = path_to_init + '/' + str(int(i / 4)) + '.wav'
        # синтез фразы
        aud_silero = model.apply_tts(text=lines[i+3], speaker=speaker, sample_rate=sample_rate,
                                put_accent=True, put_yo=True)
        silero_save(aud_silero, export_path, sample_rate)
        if speed != 1:
          speedup(export_path, speed)
    else:
      sample_rate = 16000
      for i in range(0, len(lines), 4):
        export_path = path_to_init + '/' + str(int(i / 4)) + '.wav'
        aud_silero = model.apply_tts(texts=[lines[i+3]], sample_rate=sample_rate)
        silero_save(aud_silero, export_path, sample_rate)
        if speed != 1:
          speedup(export_path, speed)

  # скачивание архива
  !zip -r synthesized1.zip synthesized1
  files.download("synthesized1.zip")

  clear_output()
  print('Синтезированная речь загружена на ваш компьютер. Вы можете прослушать её и вернуться к работе.')
  print('Вы можете, например, загрузить новый SRT файл с непонравившимися фразами и перезапустить эту ячейку.')
else:
  print('Вы не загрузили текстовый файл!')

In [ ]:
# @title Рекомендация по выбору инструмента для синтеза речи

# @markdown <font color="green"> Заполните поля ниже, и вам будут предложены подходящие варианты синтеза речи.

# @markdown Введите язык (на английском), на котором будет производиться синтез:
language = 'english' # @param {type: "string"}
# @markdown Собираетесь ли вы клонировать голос? (для повышения сходства с оригиналом)
cloning = False  # @param {type: "boolean"}
# @markdown Какой голос предпочтительнее?
gender = "мужской"  # @param ["мужской", "женский"]
# @markdown Нужно ли вам придать голосу эмоцию?
emotions = False  # @param {type: "boolean"}

# @markdown Примеры голосов Yandex можно послушать на этой странице: https://cloud.yandex.ru/ru/services/speechkit#demo

if not gtts_installed:
  !pip install gTTS
  from gtts import gTTS, lang
if not edge_installed:
  !pip install edge-tts
  import asyncio
  import edge_tts
  from edge_tts import VoicesManager

selected = False # переменная для пометки того, что хотя бы один инструмент подходит

language = language.lower()
if emotions:
  if language in yandex_languages:
    header = False
    if cloning: # Если голос будет клонироваться, пол голоса модели не важен
      for model in yandex_languages.get(language):
        if model['roles']:
          if not header:
            print('Вам могут подойти следующие модели из библиотеки Yandex SpeechKit: \n')
            header = True
          print('Имя: ', model['name'], '\nПол: ', model['gender'], '\nС возможными амплуа: ', ', '.join(model['roles'], '\n'))
    else:
      gender = 'male' if gender == 'мужской' else 'female'
      for model in yandex_languages.get(language):
        if model['gender'] == gender and model['roles']:
          if not header:
            print('Вам могут подойти следующие модели из библиотеки Yandex SpeechKit: \n')
            header = True
          print('Имя: ', model['name'], '\nС возможными амплуа: ', ', '.join(model['roles'], '\n'))
  else:
    print('К сожалению, эмоции недоступны на выбранном языке.')
else:
  # Проверка Yandex SpeechKit
  if language in yandex_languages.keys():
    header = False
    if cloning: # Если голос будет клонироваться, пол голоса модели не важен
      for model in yandex_languages.get(language):
        if not header:
          print('Вам могут подойти следующие модели из библиотеки Yandex SpeechKit: \n')
          header = True
        print('Имя: ', model['name'], '\nПол: ', model['gender'], '\nС возможными амплуа: ', ', '.join(model['roles'], '\n'))
    else:
      gender_yandex = 'male' if gender == 'мужской' else 'female'
      for model in yandex_languages.get(language):
        if model['gender'] == gender_yandex:
          if not header:
            print('Вам могут подойти следующие модели из библиотеки Yandex SpeechKit: \n')
            header = True
          print('Имя: ', model['name'], '\nС возможными амплуа: ', ', '.join(model['roles'], '\n'))
    if header:
      selected = True
  # Проверка Coqui TTS
  if cloning:
    if language == 'english':
      print('Также рассмотрите модели XTTSv2 и Tacotron2-DDC_ph из Coqui TTS.')
    elif language in xtts_languages:
      if selected:
        print('Также рассмотрите модель XTTSv2 из Coqui TTS.')
      else:
        print('Вам может подойти модель XTTSv2 из Coqui TTS.')
        selected = True
  elif language == 'english' and gender == 'женский':
    print('Вам может подойти модель Tacotron2-DDC_ph из Coqui TTS.')
    selected = True

  language = language[0].upper() + language[1 :]
  # Проверка gTTS
  lang_available = False
  for val in lang.tts_langs().values():
    if language in val:
      lang_available = True
  if lang_available:
    if selected:
      print('Также рассмотрите gTTS.')
    else:
      print('Вам может подойти синтез речи с помощью gTTS.')
      selected = True
    if not cloning:
      print('Учтите, что в gTTS нет выбора пола голоса модели.')
  # Проверка edge-tts
  lang_code = iso639.to_iso639_1(language) # получение кода языка ISO639-1
  gender_edge = 'Male' if gender == 'мужской' else 'Female'
  if not vm_crtd:
    vm = await VoicesManager.create()
    vm_crtd = True
  if cloning:
    voices = vm.find(Language = lang_code)
    if voices:
      if selected:
        print('Также рассмотрите Microsoft Edge TTS.')
        print('Вам могут подойти следующие голоса:')
      else:
        print('Вам могут подойти следующие голоса Microsoft Edge TTS:')
        selected = True
      for voice in voices:
        print(voice['ShortName'], ', пол: ', voice['Gender'])
  else:
    voices = vm.find(Gender = gender_edge, Language = lang_code)
    if voices:
      if selected:
        print('Также рассмотрите Microsoft Edge TTS.')
        print('Вам могут подойти следующие голоса:')
      else:
        print('Вам могут подойти следующие голоса Microsoft Edge TTS:')
        selected = True
      for voice in voices:
        print(voice['ShortName'])
  # Проверка Silero Models
  if language in silero_languages:
    if selected:
      print('Также можно посоветовать Silero Models.')
    else:
      print('Вам может подойти Silero Models.')
      selected = True
    print('Совет: поэкспериментируйте, выбирая голос в ячейке ниже.') #???
  if not selected:
    print('К сожалению, моделей с запрашиваемыми параметрами нет.')

In [ ]:
# @title Тестирование голосов

# @markdown Выберите синтезатор для тестирования:
tool = "gTTS" # @param ["gTTS", "Microsoft Edge TTS", "Silero Models"]

# @markdown Введите язык (для gTTS и Silero Models), на английском:
language = 'english' # @param {type: "string"}

# @markdown Введите голоса (для Microsoft Edge TTS и Silero Models):
voice_name = 'ru-RU-DmitryNeural' # @param {type: "string"}

# @markdown Выберите пол голоса (для некоторых языков Silero):
gender = 'male' # @param ["male", "female"]

# @markdown <font color="darkblue"> *Голоса Silero на выбор, которые вы можете протестировать:*

# @markdown * Русский: 'aidar', 'baya', 'kseniya', 'xenia', 'eugene'

# @markdown * Английский: en_0-en_117

# @markdown * Немецкий: 'bernd_ungerer', 'eva_k', 'friedrich', 'hokuspokus', 'karlsson'

# @markdown * Испанский: es_0-es_2

# @markdown * Французский: fr_0-fr_5

# @markdown * Калмыцкий: 'erdni', 'delghir'

# @markdown * Украинский 'aidar', 'baya', 'kseniya', 'xenia', 'eugene'

# @markdown * Для индийских языков доступны женские и мужские, кроме Manipuri (только женский).

# @markdown * Кириллические языки: 'b_bulb', 'b_bulc', 'b_tat', 'b_ru', 'b_tyv', 'b_uzb', 'b_che', 'b_kalmyk',
# @markdown 'b_sah', 'b_myv', 'b_kjh', 'b_lez', 'b_krc', 'b_cv', 'b_udm', 'b_bashkir','b_nog','b_oss', 'b_ava',
# @markdown 'b_mhr', 'b_kpv', 'b_mrj', 'kz_M1', 'kz_M2', 'kz_F3', 'kz_F1', 'kz_F2', 'cv_ekaterina', 'marat_tt',
# @markdown 'kalmyk_erdni', 'kalmyk_delghir'

# @markdown <font color="darkblue"> *Для остальных доступных языков выбора нет (одна модель).*

# @markdown Введите текст:
text = 'Hello, this is a test text.' # @param {type: "string"}


if tool == "gTTS":
  if not gtts_installed:
    !pip install gTTS
    from gtts import gTTS, lang
    gtts_installed = True
  # синтез gTTS
  lang_code = iso639.to_iso639_1(language) # получение кода языка ISO639-1
  test_aud = gTTS(text, lang=lang_code)
  test_aud.save('test_aud.mp3')
  clear_output()
  # отображение аудио для проигрывания в среде
  display(Audio('test_aud.mp3'))

elif tool == "Microsoft Edge TTS":
  if not edge_installed:
    !pip install edge-tts
    import asyncio
    import edge_tts
    from edge_tts import VoicesManager
    edge_installed = True
  # синтез edge-tts
  test_aud = edge_tts.Communicate(text, voice_name)
  await test_aud.save('test_aud.mp3')
  clear_output()
  display(Audio('test_aud.mp3'))

else:
  lang_code = iso639.to_iso639_1(language) # получение кода языка ISO639-1
  if not silero_installed:
    !pip install -q torchaudio omegaconf
    from pprint import pprint
    from omegaconf import OmegaConf
    from scipy.io import wavfile
    import numpy as np
    silero_installed = True
  # синтез Silero
  model_id, voice, ver = choice_silero_model(language, gender=gender)
  # Проверяем доступность GPU
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  if not voice and ver == '3-4':
    voice = voice_name
  model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_tts',
                                       language=lang_code,
                                       speaker=model_id)
  model.to(device)
  if ver == '3-4':
    sample_rate = 48000
    audio = model.apply_tts(text=text,
                            speaker=voice,
                            sample_rate=sample_rate,
                            put_accent=True,
                            put_yo=True)
    clear_output()
    display(Audio(test_audio, rate = sample_rate))
  else:
    sample_rate = 16000
    audio = model.apply_tts(texts=[text],
                            sample_rate=sample_rate)
    clear_output()
    display(Audio(audio[0], rate=sample_rate))

# Клонирование голоса

In [ ]:
# @title Выберите инструмент для клонирования:
tool = "Coqui freeVC24" # @param ["Coqui freeVC24", "OpenVoice"]

# проверка на наличие файла для клонирования
if os.path.isfile(clone_sample):
  if os.path.exists(path_to_init):
    if tool == "Coqui freeVC24":
      if not coqui_installed:
        # установка TTS
        !pip install TTS
        from TTS.api import TTS
        coqui_installed = True
        # Получить устройство
        device = "cuda" if torch.cuda.is_available() else "cpu"

      # загрузка модели для клонирования голоса
      tts = TTS(model_name = "voice_conversion_models/multilingual/vctk/freevc24", progress_bar = False).to(device)

      path_to_cloned = '/content/cloned'
      !mkdir $path_to_cloned
      for fn in os.listdir(path_to_init):
        # клонирование голоса
        tts.voice_conversion_to_file(source_wav = path_to_init + '/' + fn,
                                     target_wav = clone_sample,
                                     file_path = path_to_cloned + '/' + fn)
      !rm -rf $path_to_init
      os.rename(path_to_cloned, path_to_init)
    else:
      %cd /content
      !git clone https://github.com/myshell-ai/OpenVoice
      %cd /content/OpenVoice

      !pip install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda==11.7
      !pip install -r requirements.txt

      from openvoice import se_extractor
      from openvoice.api import ToneColorConverter

      !mkdir /content/OpenVoice/checkpoints
      !wget 'https://myshell-public-repo-hosting.s3.amazonaws.com/checkpoints_1226.zip' -O '/content/OpenVoice/checkpoints/checkpoints_1226.zip'
      !unzip /content/OpenVoice/checkpoints/checkpoints_1226.zip -d /content/OpenVoice/checkpoints

      ckpt_converter = '/content/OpenVoice/checkpoints/checkpoints/converter'
      device = 'cuda:0'
      tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
      tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

      path_to_cloned = '/content/cloned'
      !mkdir $path_to_cloned
      reference_speaker = clone_sample
      target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad = True)
      for fn in os.listdir('/content/synthesized1'):
        base_speaker = '/content/synthesized1/' + fn
        source_se, audio_name = se_extractor.get_se(base_speaker, tone_color_converter, vad = True)
        src_path = '/content/synthesized1/' + fn
        save_path = path_to_cloned + '/' + fn
        # клонирование голоса
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path = src_path,
            src_se = source_se,
            tgt_se = target_se,
            output_path = save_path,
            message = encode_message)
      !rm -rf $path_to_init
      os.rename(path_to_cloned, path_to_init)
      used_openvoice = True
      !zip -r synthesized1.zip synthesized1 # на всякий случай
    download = input('Нужно ли сохранить zip-архив с результатами? y/n ')
    if download == 'y':
      # скачивание архива
      !zip -r synthesized1.zip synthesized1
      files.download("synthesized1.zip")
    clear_output()
  else:
    print('Отсутствует папка с синтезированной речью. Похоже, что вы не синтезировали речь, либо не загрузили её в среду.')
else:
  print('Вы не загрузили образец голоса для клонирования. Пожалуйста, вернитесь ко второму разделу.')

# Замена аудиодорожки в видео на синтезированную

In [ ]:
# @title Нарезка видео для второго варианта замены аудиодороджки (может длиться час и более)

# @markdown Скачать zip-архивы с получившимися фрагментами (2 шт.)?
download_file = False # @param {type: "boolean"}

if used_openvoice == True:
  !pip install ffmpeg-python
  import ffmpeg
  path_to_video = '/content/vid.mp4'
  path_to_text = '/content/subtitles.srt'
path_to_fragments = '/content/fragments'
path_to_intermediate = '/content/int_fragms'
!mkdir /content/fragments
!mkdir /content/int_fragms

if os.path.isfile(path_to_video) and os.path.isfile(path_to_text):
  # считываем из файла строки
  txtfile = open(path_to_text, 'r', encoding = 'utf-8')
  lines = [''] + txtfile.read().split('\n')
  txtfile.close()

  # копируем заставку
  !ffmpeg -y -i $path_to_video -to {str_to_time(lines[2][:lines[2].find(' ')])} -async 1 /content/int_fragms/s.mp4

  for i in range(0, len(lines), 4):
    out_path1 = path_to_fragments + '/' + str(int(i / 4)) + '.mp4'
    out_path2 = path_to_intermediate + '/' + str(int(i / 4)) + '.mp4'
    start = str_to_time(lines[i + 2][:lines[i + 2].find(' ')]) # в секундах, float
    end = str_to_time(lines[i + 2][lines[i + 2].rfind(' ') + 1:])
    # копирование части видео по таймингу
    !ffmpeg -y -i $path_to_video -ss {start + 0.001} -to {end} -async 1 $out_path1
    if i != len(lines) - 4: # не последний тайминг
      next_start = str_to_time(lines[i + 6][:lines[i + 6].find(' ')])
      if next_start - end > 0:
        !ffmpeg -y -i $path_to_video -ss {end + 0.001} -to {next_start} -async 1 $out_path2
    else:
      if round(video_duration(path_to_video), 3) > end:
        !ffmpeg -y -i $path_to_video -ss {end + 0.001} -async 1 $out_path2
  clear_output()

  print('Фрагменты успешно нарезаны. После этого в нижней ячейке выберите второй вариант.')
  if download_file:
    !zip -r /content/fragments.zip /content/fragments
    !zip -r /content/int_fragms.zip /content/int_fragms
    files.download("/content/fragments.zip")
    files.download("/content/int_fragms.zip")
    clear_output()
    print('Они будут автоматически загружены в zip-архивах на ваш компьютер. Если собираетесь продолжить позже, выгрузите их в соответсвующей ячейке раздела загрузок.')
else:
  print('Похоже, вы не загрузили видео или текстовый файл (или всё вместе).')

In [ ]:
# @title Выберите способ:
mode = "Простое наложение аудио по таймингам" # @param ["Простое наложение аудио по таймингам", "Растянуть/сузить видео, чтобы подходило под фразы"]
# @markdown Замедлять фразы для лучшего совпадения со временем произношения? (для 1 варианта)
slow_aud = False # @param {type: "boolean"}
# @markdown Ограничивать замедление/ускорение? (для обоих вариантов) Это нужно для того, чтобы не было слишком большой разницы между скоростями фраз или видеофрагментов в итоговом видео, если синтезированная фраза намного длиннее/короче оригинальной.
limit = True # @param {type: "boolean"}
# @markdown Скачать результаты?
download_file = True # @param {type: "boolean"}

ok = True

if os.path.exists(path_to_init) and os.path.isfile(path_to_text):
  !mkdir $path_to_synth

  # чтение файла
  txtfile = open(path_to_text, "r", encoding = "utf-8")
  lines = [''] + txtfile.read().split('\n')
  txtfile.close()

  if mode == "Простое наложение аудио по таймингам":
    if os.path.isfile(path_to_video):
      path_to_screensaver = '/content/s.mp4'
      # вырезание заставки из видео
      !ffmpeg -y -i $path_to_video -to {str_to_time(lines[2][:lines[2].find(' ')])} -async 1 $path_to_screensaver

      for i in range(0, len(lines), 4):
        old_path = path_to_init + '/' + str(int(i / 4)) + '.wav'
        new_path = path_to_synth + '/' + str(int(i / 4)) + '.wav'
        # изменение продолжительности аудио в соответствии с таймингом
        check_duration(old_path, new_path, str_to_time(lines[i+2][:lines[i+2].find(' ')]),
                       str_to_time(lines[i+2][lines[i+2].rfind(' ') + 1:]), slow_aud, limit)
        # добавление паузы после фразы до следующего тайминга или до конца
        audio = AudioSegment.from_wav(new_path)
        aud_len = len(audio) / 1000
        if i != len(lines) - 4: # если не последний тайминг
          # продолжительность от начала текущего сегмента до начала следующего (в секундах)
          fragm_dur = str_to_time(lines[i+6][:lines[i+6].find(' ')]) - str_to_time(lines[i+2][:lines[i+2].find(' ')])
          if aud_len < fragm_dur:
            add_pause(new_path, new_path, fragm_dur - aud_len)
        else: # последний тайминг
          # продолжительность от начала текущего сегмента до конца видео (в секундах)
          fragm_dur = video_duration(path_to_video) - str_to_time(lines[i+2][:lines[i+2].find(' ')])
          if aud_len < fragm_dur:
            add_pause(new_path, new_path, fragm_dur - aud_len)
    else:
      print('Вы не загрузили видео. Необходимо сделать это в разделе загрузок.')
      ok = False

  else:
    path_to_fragments = '/content/fragments'
    path_to_intermediate = '/content/int_fragms'
    path_to_screensaver = '/content/int_fragms/s.mp4'

    if (os.path.exists(path_to_fragments) and os.path.exists(path_to_intermediate)):
      for i in range(0, len(lines), 4):
        old_path = path_to_init + '/' + str(int(i / 4)) + '.wav'
        new_path = path_to_synth + '/' + str(int(i / 4)) + '.wav'
        vid_path = path_to_fragments + '/' + str(int(i / 4)) + '.mp4'
        pause_path = path_to_intermediate + '/' + str(int(i / 4)) + '.mp4'

        # замедление/ускорение видеофрагмента, если синтезированная фраза длиннее/короче
        check_vid_duration(old_path, vid_path, limit)
        if os.path.exists(pause_path): # есть следующий фрагмент видео (тайминги не одинаковые для конца и начала след. фразы)
            # добавление паузы, равной продолжительности следующего видеофрагмента
            add_pause(old_path, new_path, video_duration(pause_path))
        else: # иначе всё равно копируем в новую папку
          !cp $old_path $new_path
    else:
      print('Вы не нарезали видео на фрагменты. Это можно сделать в ячейке выше. Или загрузите фрагменты в разделе загрузок.')
      ok = False

  if ok:
    # склеиваем все фразы в целую аудиодорожку
    synthesized = [path_to_synth + '/' + item for item in os.listdir(path_to_synth) if '.ipynb' not in item]
    synthesized = sorted(synthesized, key = extract_number)
    start = str_to_time(lines[2][:lines[2].find(' ')])
    if start: # фраза начинается не с самого начала
      combined = AudioSegment.silent(duration = start * 1000) # тишина
    else:
      combined = None
    for fragment in synthesized:
      audio = AudioSegment.from_file(fragment, format = "wav")
      if combined is None:
          combined = audio
      else:
          combined += audio
    # сохранение результата
    combined.export('/content/synthesized_speech.wav', format = "wav")

    # Наложение на видео новой аудиодорожки
    if mode == "Растянуть/сузить видео, чтобы подходило под фразы":
      # объединение изменённых видеофрагментов
      fragments = [path_to_fragments + '/' + item for item in os.listdir(path_to_fragments) if '.ipynb' not in item]
      fragments = sorted(fragments, key = extract_number)
      int_fragms = [path_to_intermediate + '/' + item for item in os.listdir(path_to_intermediate) if '.ipynb' not in item and 's' not in item]
      int_fragms = sorted(int_fragms, key = extract_number)
      loaded_video_list = []
      loaded_video_list.append(VideoFileClip(path_to_intermediate + '/s.mp4')) # заставка
      for i, vid in enumerate(fragments):
        loaded_video_list.append(VideoFileClip(vid))
        next_fragm = path_to_intermediate + '/' + str(i) + '.mp4'
        if os.path.exists(next_fragm): # если существует промежуточный фрагмент
          loaded_video_list.append(VideoFileClip(next_fragm))
      concatenate_clip = concatenate_videoclips(loaded_video_list)
      if os.path.exists(path_to_video):
        os.remove(path_to_video)
      concatenate_clip.write_videofile(path_to_video, logger = None)

      # исправление файла с субтитрами с помощью списка times
      subs_path = '/content/new_subs.srt'
      for i in range(0, len(lines), 4):
        audio_length = times[int(i / 4)] # в секундах
        if i == 0: # первый тайминг
          start_timing = lines[i + 2][:lines[i + 2].find(' ')]
          end_timing = time_to_str(str_to_time(start_timing) + audio_length)
          lines[i + 2] = start_timing + ' --> ' + end_timing
          pause = path_to_intermediate + '/' + str(int(i / 4)) + '.mp4'
          if os.path.exists(pause):
            start_timing = time_to_str(str_to_time(end_timing) + video_duration(pause))
          else:
            start_timing = end_timing
        elif i == len(lines) - 4: # последний
          end_timing = time_to_str(str_to_time(start_timing) + audio_length)
          lines[i + 2] = start_timing + ' --> ' + end_timing
        else: # средний
          end_timing = time_to_str(str_to_time(start_timing) + audio_length)
          lines[i + 2] = start_timing + ' --> ' + end_timing
          pause = path_to_intermediate + '/' + str(int(i / 4)) + '.mp4'
          if os.path.exists(pause):
            start_timing = time_to_str(str_to_time(end_timing) + video_duration(pause))
          else:
            start_timing = end_timing

      txt_file = open(subs_path, 'w', encoding = "utf-8")
      for line in lines[1:-1]:
        txt_file.write(line + '\n')
      if len(lines) != 1:
        txt_file.write(lines[-1])
      txt_file.close()

    # добавление в начало аудиодорожки заставки на случай, если она была музыкальной
    screensaver = VideoFileClip(path_to_screensaver)
    music = screensaver.audio
    music.write_audiofile('/content/music.wav')
    full_audio = AudioSegment.from_wav('/content/synthesized_speech.wav')
    full_audio = AudioSegment.from_wav('/content/music.wav') + full_audio[screensaver.duration * 1000:]
    full_audio.export('/content/synthesized_speech1.wav', format = 'wav')

    # Наложение на видео аудиодорожки
    !ffmpeg -i /content/vid.mp4 -i /content/synthesized/synthesized_speech1.wav -c:v copy -c:a aac -strict experimental -map 0:v:0 -map 1:a:0 /content/result.mp4

    clear_output()

    if download_file:
      files.download("/content/result.mp4")
      if os.path.isfile("/content/new_subs.srt"):
        files.download("/content/new_subs.srt")
      clear_output()
    print('Поздравляем! Всё готово. Результаты находятся по пути: /content/result.mp4 - видео, /content/new_subs.srt - новые субтитры (если был выбран второй способ).')
else:
  if not os.path.exists(path_to_init):
    print('Не найдена папка с синтезированной речью. Возможно, вы её не синтезировали. Или вы не загрузили архив.')
  else:
    print('Вы не загрузили текст. Вы можете сделать это в разделе загрузок.')

# Размонтировать Google Drive

In [ ]:
drive.flush_and_unmount()